In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

In [2]:
# Read in CSV file
regional_df = pd.read_csv("Table_CSVs/regional.csv")
regional_df

,DOEID,REGIONC,DIVISION,state_postal,BA_climate,TOTALBTU,TOTALDOL
0,100001,WEST,Mountain South,NM,Mixed-Dry,144647.71,2656.89
1,100002,SOUTH,West South Central,AR,Mixed-Humid,28034.61,975.00
2,100003,WEST,Mountain South,NM,Mixed-Dry,30749.71,522.65
3,100004,SOUTH,South Atlantic,SC,Mixed-Humid,86765.19,2061.77
4,100005,NORTHEAST,Middle Atlantic,NJ,Mixed-Humid,59126.93,1463.04
...,...,...,...,...,...,...,...
18491,118492,SOUTH,South Atlantic,MD,Mixed-Humid,49930.49,1098.51
18492,118493,NORTHEAST,New England,ME,Very-Cold,222186.04,3613.44
18493,118494,SOUTH,West South Central,TX,Hot-Humid,51593.72,1428.31
18494,118495,SOUTH,South Atlantic,SC,Hot-Humid,63555.21,2224.94


In [5]:
# Create DataFrame with regional information
regional_df = regional_df.drop(columns='DOEID')
regional_df.head()

,REGIONC,DIVISION,state_postal,BA_climate,TOTALBTU,TOTALDOL
0,WEST,Mountain South,NM,Mixed-Dry,144647.71,2656.89
1,SOUTH,West South Central,AR,Mixed-Humid,28034.61,975.00
2,WEST,Mountain South,NM,Mixed-Dry,30749.71,522.65
3,SOUTH,South Atlantic,SC,Mixed-Humid,86765.19,2061.77
4,NORTHEAST,Middle Atlantic,NJ,Mixed-Humid,59126.93,1463.04


In [6]:
#Checking nulls
regional_df.isnull().sum()

REGIONC         0
DIVISION        0
state_postal    0
BA_climate      0
TOTALBTU        0
TOTALDOL        0
dtype: int64

In [7]:
# One-hot encode the categorical features and scale the numerical targets
ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(sparse=False), ['REGIONC', 'DIVISION', 'state_postal', 'BA_climate'])],
    remainder='passthrough'
)

In [8]:
# Prepare the features
features = ['REGIONC', 'DIVISION', 'state_postal', 'BA_climate']
X = regional_df[features]
X = ct.fit_transform(X)

c:\ProgramData\anaconda3\envs\dev\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [9]:
# Prepare the targets
target_features = ['TOTALDOL', 'TOTALBTU']
Y = regional_df[target_features]
Y = StandardScaler().fit_transform(Y)  

In [11]:
# Split the dataset into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [12]:
# Define the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dense(Y_train.shape[1])  # Output layer with 2 neurons (one for each target)
])

In [13]:
# Compile the model
model.compile(optimizer=Adam(), loss='mse')

In [14]:
# Fit the model to the data
model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100

370/370 [==============================] - 1s 1ms/step - loss: 0.9401 - val_loss: 0.8838
Epoch 2/100
370/370 [==============================] - 0s 1ms/step - loss: 0.9198 - val_loss: 0.8837
Epoch 3/100
370/370 [==============================] - 0s 1ms/step - loss: 0.9174 - val_loss: 0.8813
Epoch 4/100
370/370 [==============================] - 0s 1ms/step - loss: 0.9147 - val_loss: 0.8845
Epoch 5/100
370/370 [==============================] - 1s 1ms/step - loss: 0.9147 - val_loss: 0.8807
Epoch 6/100
370/370 [==============================] - 0s 1ms/step - loss: 0.9144 - val_loss: 0.8799
Epoch 7/100
370/370 [==============================] - 1s 2ms/step - loss: 0.9130 - val_loss: 0.8788
Epoch 8/100
370/370 [==============================] - 1s 1ms/step - loss: 0.9129 - val_loss: 0.8835
Epoch 9/100
370/370 [==============================] - 0s 1ms/step - loss: 0.9132 - val_loss: 0.8819
Epoch 10/100
370/370 [==============================] - 0s 1ms/step - loss: 0.9121 - val_l

In [16]:
# Evaluate the model on the test data
test_loss = model.evaluate(X_test, Y_test)

116/116 [==============================] - 0s 784us/step - loss: 0.8223


In [17]:
# Predict the energy consumption for the test set
Y_pred = model.predict(X_test)

116/116 [==============================] - 0s 678us/step


In [18]:
#Inverse the scaling to interpret the predictions in their original scale
scaler = StandardScaler().fit(regional_df[target_features])
Y_pred_rescaled = scaler.inverse_transform(Y_pred)

In [20]:
print("Test loss (MSE):", test_loss)
print("Predictions:", Y_pred_rescaled)

Test loss (MSE): 0.8223126530647278
Predictions: [[  2286.1555  81924.79  ]
 [  1932.0144 103277.48  ]
 [  1987.1519  93124.64  ]
 ...
 [  2145.0022  91796.49  ]
 [  1939.3438 101472.    ]
 [  2106.3545 104262.85  ]]


In [22]:
# Calculate R-squared
r2 = r2_score(Y_test, Y_pred_rescaled)
print("R-squared:", r2)

NameError: name 'r2_score' is not defined